Import Libraries

In [1]:
import pandas as pd
import polars as pl
import datetime
from datetime import datetime, timedelta
import os
import s3fs
import boto3
from io import BytesIO as bo

Calculate Date Variables

In [2]:
#variables -
this_day = datetime.today()
# ### FOR TESTING - REMOVE LATER ### 
this_day = this_day - timedelta(days=7)
# ####
days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)

CUR_PROC_WK = monday.strftime("%Y%m%d")

CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)
QTR = f'{this_day.year}Q{(this_day.month-1)//3+1}'

XPN_D = CUR_WK[4:]

In [3]:
# Grabbing IW Release File 
s3 = s3fs.S3FileSystem()
bucket_list = s3.listdir(f'vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/inbound/')
for file in bucket_list:
    if (file['Key'].__contains__(f"IRWD_RELEASE_WKLY_{CUR_PROC_WK}")):
        release_file = file['Key']

# Reading Release File -
s3_client = boto3.client('s3')
bucket = release_file.split('/')[0]
file_key = release_file.split('/',1)[1]

adm = pd.read_csv(f's3://{bucket}/{file_key}',sep='|')
adm1 = adm[CUR_PROC_WK].tolist()

for item in adm1:
    if item.startswith('P_OUT_110_CUST-DECILE_'):
        DECILE_TIMESTAMP = item.split('_')[-1].split('.')[0]
    elif item.startswith('P_OUT_110_CUST-TARGET_'):
        TARGET_TIMESTAMP = item.split('_')[-1].split('.')[0]

Libraries and Paths

In [4]:
#LIBNAMES
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
staging = 'PYADM/weekly/staging/reference/'
ag = f'PYADM/quaterly/{QTR}/target/post/'

In [5]:
# Reading Raw Data (Consider reading in polars to begin with ? may save a few micro seconds)

CUST_DECILE = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_CUST-DECILE_{DECILE_TIMESTAMP}.TXT',delimiter='|',dtype={1:str,2:str,5:str},parse_dates=[6,7]
)
#CUST_DECILE['DecileID'] = CUST_DECILE['DecileID'].fillna("") #Consider Removing this to save space long term  ? (None takes less space than "")
CUST_TARGET = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_CUST-TARGET_{TARGET_TIMESTAMP}.TXT',delimiter='|',dtype={1:str,9:str},parse_dates=[7,8]
)

#CUST_TARGET[['TargetID', 'TargetFlagValue']] = CUST_TARGET[['TargetID', 'TargetFlagValue']].fillna("")

# Converting to polars for faster Operations  - 
CUST_DECILE = pl.from_pandas(CUST_DECILE)
CUST_TARGET = pl.from_pandas(CUST_TARGET)

In [6]:
DECILE_TARGET = CUST_DECILE.join(CUST_TARGET,on='IID',how='left')
#Note nobs increased after left join , one to many relation posible

# Exporting -
#DECILE_TARGET.write_parquet(staging+'\\target_decile_'+XPN_D+'.parquet',compression='zstd',compression_level=10,use_pyarrow=True)
DECILE_TARGET.to_pandas().to_parquet(f's3://{bucket}/{staging}target_decile_{XPN_D}.parquet')

### Abbvie Targets Code -

In [7]:
print(DECILE_TARGET.group_by('TargetName').len())

shape: (3, 2)
┌──────────────────┬─────────┐
│ TargetName       ┆ len     │
│ ---              ┆ ---     │
│ str              ┆ u32     │
╞══════════════════╪═════════╡
│ LINZESS          ┆ 13112   │
│ FRX Sales Target ┆ 36069   │
│ null             ┆ 1697491 │
└──────────────────┴─────────┘


In [8]:
TD = DECILE_TARGET.filter(pl.col('TargetName')=='FRX Sales Target')

if (TD.shape[0] != 0):
    TD = TD.sort('IID')
    #TD.write_parquet(ag+'\\ABBVIE_TARGET.parquet',compression='zstd',compression_level=10,use_pyarrow=True)
    TD.to_pandas().to_parquet(f's3://{bucket}/{ag}ABBVIE_TARGET.parquet',compression='snappy')
else:
    print('Not Exporting since no new FRX Sales Target')
    
#Not adding export to bit location as it might exist after transition - 'U:\Ironwood\projects\BIT\2023Q4TargetPost'